In [1]:
import glob, os
import numpy as np
import math, random

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

from geometry.model import Model, combine_observations, get_mesh
from geometry.utils.visualisation import illustrate_points, illustrate_mesh, illustrate_voxels
from geometry.voxel_grid import VoxelGrid

from rl.environment import Environment, CombiningObservationsWrapper
from rl.environment import StepPenaltyRewardWrapper, DepthMapWrapper
from rl.environment import VoxelGridWrapper, VoxelWrapper
from rl.environment import FrameStackWrapper, ActionMaskWrapper
from rl.environment import MeshReconstructionWrapper
from rl.validation import validate
from rl.utils import build_epsilon_func, plot


from rl.dqn import CnnDQN, CnnDQNA, VoxelDQN
from rl.agent import DQNAgent
from rl.replay_buffer import DiskReplayBuffer, ReplayBuffer


# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [2]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

experiment_save_path = "./models/abc-vdqn-occl-sgd/"

train_dataset_path = "./data/1kabc/simple/train/"
val_dataset_path = "./data/1kabc/simple/val/"
number_of_view_points = 100

num_stack = 4
reconstruction_depth = 7
grid_size = 64
raycast_resolution = 512


learning_rate = 0.001
weight_decay = 0.01
buffer_capacity = 100000
epsilon_decay = 10000
batch_size = 256
start_frame = 0
num_frames = 150000

log_interval = 100
save_interval = 500
val_interval = 1000
train_interval = 10
max_novp = 50

In [4]:
env = Environment(models_path=train_dataset_path,
#                   similarity_threshold=similarity_threshold,
                  image_size=raycast_resolution,
                  number_of_view_points=number_of_view_points)
# env = CombiningObservationsWrapper(env)
# env = StepPenaltyRewardWrapper(env, weight=1.0)
# env = DepthMapWrapper(env)

env = MeshReconstructionWrapper(env, reconstruction_depth=reconstruction_depth)
env = VoxelGridWrapper(env, occlusion_reward=True, grid_size=grid_size)
env = CombiningObservationsWrapper(env)
env = VoxelWrapper(env)
env = StepPenaltyRewardWrapper(env)
env = FrameStackWrapper(env, num_stack=num_stack, lz4_compress=False)
env = ActionMaskWrapper(env)

In [4]:
agent = DDQNAgent(env.observation_space.shape, env.action_space.n,
                  device=device, learning_rate=learning_rate, weight_decay=weight_decay)

replay_buffer = DiskReplayBuffer(capacity=buffer_capacity,
                                 overwrite=True,
                                 location="buffer_voxels/",
                                 num_actions=env.action_space.n,
                                 observation_dtype=env.observation_space.dtype,
                                 observation_shape=env.observation_space.shape)

epsilon_by_frame = build_epsilon_func(epsilon_decay=epsilon_decay)

### Training

1. + Validation
2. + Saver
3. + Save loss plot
4. Config saver
5. Starter

In [ ]:
if not os.path.exists(experiment_save_path):
    os.makedirs(experiment_save_path)
else:
#     model = torch.load(save_model_path.format(start_frame)).to(device)    
    pass


losses, all_rewards, all_nofs = [], [], []
episode_reward = 0
nof_vp = 0
best_metric = 0

state, _, mask = env.reset()
for frame_idx in range(start_frame + 1, num_frames + 1):
    epsilon = epsilon_by_frame(frame_idx)
    action = agent.act(state, mask, epsilon)

    next_state, reward, done, _, mask = env.step(action)
    replay_buffer.push(state, action, reward, next_state, done, mask)

    state = next_state
    episode_reward += reward
    nof_vp += 1

    if done or nof_vp > max_novp:
        # if done: final_reward = env.final_reward()
        print("Frame: ", frame_idx, "Number of View Points: ", nof_vp)
        print()

        state, _, mask = env.reset()
        all_rewards.append(episode_reward)
        all_nofs.append(nof_vp)
        episode_reward = 0
        nof_vp = 0
        
    if frame_idx % train_interval == 0 and frame_idx > batch_size:
        batch = replay_buffer.sample(batch_size)
        loss = agent.compute_td_loss(*batch)
        losses.append(loss)
        
    if frame_idx % log_interval == 0:
        save_path = os.path.join(experiment_save_path, 'loss.png')
        plot(save_path, frame_idx, all_rewards, all_nofs, losses)
        
    if frame_idx % save_interval ==  0:
        for f in glob.glob(os.path.join(experiment_save_path, "last-*.pt")): os.remove(f)
        save_path = os.path.join(experiment_save_path,
                                 'last-{}.pt'.format(frame_idx))
        torch.save(agent.model, save_path)
        
    if frame_idx % val_interval == 0:
        reward, hausdorff, novp = validate(agent, models_path=val_dataset_path)
        print ("Validation metrics: ", reward, hausdorff, novp)
        if novp > best_metric:
            best_metric = novp
            save_path = os.path.join(experiment_save_path,
                                 'best-{}-{:.2f}.pt'.format(frame_idx, best_metric))
            torch.save(agent.model, save_path)


  0%|          | 0/72 [00:00<?, ?it/s]

Action:  10
0.21993170290764827 0.26628065842357207 0.4862123613312203
0.26628065842357207
Action:  35
0.24371013910510722 0.38094913189330804 0.6246592709984152
0.38094913189330804
Action:  88
0.2324360970220459 0.5756780075738336 0.8081141045958795
0.5756780075738336
Action:  97
0.18954954050118014 0.6546342946810703 0.8441838351822504
0.6546342946810703
Action:  61
0.1999447597047155 0.6804355889482163 0.8803803486529318
0.6804355889482163
Action:  78
0.2682418520564455 0.6172732034110664 0.8855150554675119
0.6172732034110664
Action:  98
0.19459649475217194 0.7039138063571784 0.8985103011093503
0.7039138063571784
Action:  59
0.10111484959574148 0.7987266717988702 0.8998415213946117
0.7987266717988702
Action:  99
0.20825591322251796 0.6923463054779575 0.9006022187004754
0.6923463054779575
Action:  46
0.06352633957716065 0.8415766715163416 0.9051030110935023
0.8415766715163416
Action:  22 (random)
0.27625169487269624 0.6565533764426761 0.9328050713153724
0.6565533764426761
Action:  95

  1%|▏         | 1/72 [00:11<13:03, 11.04s/it]

Action:  10
0.1321186440677966 -0.0537886241870809 0.07833001988071571
-0.0537886241870809
Action:  84
0.15728813559322033 0.0192526198739765 0.17654075546719683
0.0192526198739765
Action:  46
0.07254237288135593 0.11632442632341544 0.18886679920477137
0.11632442632341544
Action:  44
0.021271186440677967 0.17554789904639956 0.19681908548707752
0.17554789904639956
Action:  42
0.08483050847457627 0.13027883546180544 0.2151093439363817
0.13027883546180544
Action:  53
0.04135593220338983 0.18965798429760422 0.23101391650099404
0.18965798429760422
Action:  34
0.10516949152542372 0.15506907706304546 0.26023856858846917
0.15506907706304546
Action:  27
0.07677966101694915 0.2083098022037268 0.28508946322067596
0.2083098022037268
Action:  23
0.09686440677966102 0.2259984162819692 0.3228628230616302
0.2259984162819692
Action:  61
0.04347457627118644 0.30404033426559285 0.3475149105367793
0.30404033426559285
Action:  49
0.04169491525423729 0.31635677460659767 0.35805168986083497
0.316356774606597

  3%|▎         | 2/72 [00:15<10:32,  9.04s/it]

...

Action:  61
0.07339867791201277 0.26899212242184606 0.3423908003338588
0.26899212242184606
Action:  50
0.264037687105843 0.36908853008788783 0.6331262171937309
0.36908853008788783
Action:  10
0.08415014056682622 0.6061957742991665 0.6903459148659927
0.6061957742991665
Action:  47
0.23649418737178027 0.505785326678113 0.7422795140498933
0.505785326678113
Action:  42
0.2541600182356964 0.508058288358016 0.7622183065937124
0.508058288358016
Action:  84
0.23805181977053416 0.5505969792649847 0.7886487990355189
0.5505969792649847
Action:  46
0.26164425195653823 0.6054613772746591 0.8671056292311973
0.6054613772746591
Action:  53
0.15496542815895448 0.7222487051991091 0.8772141333580636
0.7222487051991091
Action:  51
0.2556416685662184 0.6408620873458655 0.8965037559120839
0.6408620873458655
Action:  35
0.15295190335080922 0.7561642980773647 0.909116201428174
0.7561642980773647
Action:  34
0.23846972114580958 0.6734286373815019 0.9118983585273115
0.6734286373815019
Action:  54 (random)
0.254

 89%|████████▉ | 64/72 [12:46<01:52, 14.10s/it]

Action:  46
0.1971183153228041 0.1473938797991471 0.3445121951219512
0.1473938797991471
Action:  50
0.19781102798559158 0.3519857199818881 0.5497967479674797
0.3519857199818881
Action:  44
0.06433915211970075 0.535728598557806 0.6000677506775067
0.535728598557806
Action:  99
0.0916320310335273 0.5795061803485866 0.6711382113821138
0.5795061803485866
Action:  61
0.08362427265170408 0.6219990335813583 0.7056233062330624
0.6219990335813583
Action:  47
0.1918814075921308 0.5520210314322594 0.7439024390243902
0.5520210314322594
Action:  95
0.10967026877251317 0.6695980239104137 0.7792682926829269
0.6695980239104137
Action:  81 (random)
0.19745081740094209 0.6175220823280553 0.8149728997289973
0.6175220823280553
Action:  48
0.08376281518426157 0.7478767512114024 0.8316395663956639
0.7478767512114024
Action:  40
0.10440565253532835 0.7537244287654847 0.858130081300813
0.7537244287654847
Action:  78
0.11742865059573289 0.7562840865316384 0.8737127371273713
0.7562840865316384
Action:  28 (rando

 90%|█████████ | 65/72 [12:56<01:29, 12.81s/it]

Action:  10
0.2118491381898985 0.21654272872322536 0.42839186691312386
0.21654272872322536
Action:  53
0.09419854663383581 0.37596781196135826 0.47016635859519407
0.37596781196135826
Action:  46
0.0561527836165996 0.44654592248321556 0.5026987060998152
0.44654592248321556
Action:  50
0.05840490060657018 0.47061543211062024 0.5290203327171904
0.47061543211062024
Action:  61
0.12155426100534503 0.4448782713421595 0.5664325323475046
0.4448782713421595
Action:  51
0.07425980421596301 0.5215997151925398 0.5958595194085028
0.5215997151925398
Action:  54
0.08260765119212059 0.5525124966821863 0.6351201478743068
0.5525124966821863
Action:  99
0.19365203291093627 0.5094163589559769 0.7030683918669132
0.5094163589559769
Action:  44
0.09660080475647108 0.6294620418239356 0.7260628465804067
0.6294620418239356
Action:  48
0.08516005044742057 0.6454129624915813 0.7305730129390019
0.6454129624915813
Action:  78
0.18353252056933517 0.5645635977301102 0.7480961182994454
0.5645635977301102
Action:  42
0

In [54]:
def compute_metrics(env, iter_cnt=10, max_iter=30):
    rewards, final_rewards, novp = [], [], []
    for _ in range(iter_cnt):
        state, action, mask = env.reset()
        episode_reward = 0.0
        for t in range(max_iter):
#             action = model.act(state, mask, epsilon=0.0)
            

            s   = autograd.Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), volatile=True).cuda()
            m = autograd.Variable(torch.FloatTensor(np.float32(mask)), volatile=True).cuda()
            q_value = model.forward(s)
            q_value *= m
            a = torch.nn.functional.softmax(q_value)
            
            action = a.argmax().item()
            print(action)

            state, reward, done, info, mask = env.step(action)
            # print("REWARD: ", reward)
#             env.render(action, state)
            episode_reward += reward

            if done:
                break

        final_reward = 0
        # final_reward = env.final_reward()
        # episode_reward += 1.0 / final_reward
        rewards.append(episode_reward)
        final_rewards.append(final_reward)
        novp.append(t + 1)
    return np.mean(rewards), np.mean(final_rewards), np.mean(novp)

In [55]:
agent_func = lambda s : model.act(s, epsilon=0.0)
result = compute_metrics(env)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


75
0.45846322352346447 0.007501328236103999 0.46596455175956847
0.007501328236103999
36
0.38924930491195553 0.41681285794445344 0.806062162856409
0.41681285794445344
60
0.4547982138343584 0.36333690047337336 0.8181351143077318
0.36333690047337336
33
0.4140197152245345 0.46807635464446107 0.8820960698689956
0.46807635464446107
83
0.4585896031679164 0.42350646670107917 0.8820960698689956
0.42350646670107917
59
0.4459937652708737 0.44445062209105796 0.8904443873619317
0.44445062209105796
85
0.4557249978936726 0.4388293303878583 0.8945543282815309
0.4388293303878583
98
0.45475608728620776 0.440825726225223 0.8955818135114307
0.440825726225223
13
0.476535512680091 0.4916124451930146 0.9681479578731056
0.4916124451930146
75
0.4717649647887324 0.08067738581875583 0.5524423506074883
0.08067738581875583
36
0.4639964788732394 0.19543818514858052 0.6594346640218199
0.19543818514858052
60
0.4647887323943662 0.24739822520543547 0.7121869575998017
0.24739822520543547
33
0.46307218309859155 0.2692610

0.4856278366111952 0.45664858615303244 0.9422764227642276
0.45664858615303244
88
0.4876449823499748 0.4554444485443342 0.943089430894309
0.4554444485443342
79
0.4868885526979324 0.45620087819637656 0.943089430894309
0.45620087819637656
95
0.402672718103883 0.5412297209205073 0.9439024390243902
0.5412297209205073
9
0.47402924861321233 0.46987319041117787 0.9439024390243902
0.46987319041117787
75
0.4959910430511413 0.3945139414234461 0.8905049844745874
0.3945139414234461
36
0.4826278532216122 0.4654028707528934 0.9480307239745056
0.4654028707528934
60
0.48949003178272177 0.47455638102982933 0.9640464128125511
0.47455638102982933
75
0.4495060373216246 0.2607704793931918 0.7102765167148164
0.2607704793931918
36
0.4339553604098061 0.2895279247738506 0.7234832851836567
0.2895279247738506
60
0.4606659348701061 0.27602411878239086 0.736690053652497
0.27602411878239086
33
0.42993047932674716 0.3174900737066824 0.7474205530334296
0.3174900737066824
83
0.4610318331503842 0.3074370071302597 0.7684

In [56]:
result

(-6.88126904400267, 0.0, 12.2)

In [151]:
print((6 * 256 * 256 * 8) / 1e9 * 3000, 'G')
print((10 * 512 * 512 * 8) / 1e9 * 5000, 'G')
print(1e6 * 10 * 512 * 512 * 4 / 2 ** 40, 'T')
print(1e5 * 64 * 64 * 64 * 4 * 4 / 2 ** 40, 'T')

9.437184 G
104.8576 G
9.5367431640625 T
0.3814697265625 T


In [ ]:
TODO Today:
    1. + check DQN with A, S as input -> send Sergey
    2. + Smaller depth_maps (6 * 256 * 256)
    3. + Illustrate reward as area + novp
    4. + Distributed Buffer
    5. + Buffer on hard disk (memmap)
    6. Floats to int
    7. GPU raycasting
    8. * Greedy algo
    9. * Voxels
    10. CNN different shapes input
    11. Preprocessed depth_maps
    
    10. Overfit experiments 
    11. ABC experiment
    
    12. Experiment radius = 1.0
    13. Experiments with smaller reward fine
    14. Experiment with different DQN-s
    15. ABC - write random model reading
    

Ideas:
    1. Add fine for same view_point
    
Big Experiments:
    1. Voxels
    2. PointNet
    3. Context (N first view_points)
    4. Meta-learning, подавать от простой к сложной детальке

In [15]:
difficult = [
    "00020107_b27a1602d1d44a3d89140ce4_007.obj",
    "00010095_5ae1ee45b583467fa009adc4_006.obj",
    "00010163_ccef4063b69f428e91b498c9_008.obj",
    "00010145_77759770d8cd48af80775d86_002.obj",
    "00010153_556de37e0a7447fcbfbdfd22_000.obj",
    "00010162_ccef4063b69f428e91b498c9_007.obj",
    "00020074_37170a1ba80747f1a1478985_000.obj",
    "00020077_bf146f0c5dee4199be920a21_000.obj",
    "00020080_7a689565e1e0481ca3ad4a6f_000.obj",
    "00010164_ccef4063b69f428e91b498c9_009.obj",
    "00010179_f91d806ac1e34ea1b14e23be_000.obj",
    "00020095_842a932142a9431784488344_000.obj",
    "00020097_e24ecc9c647f4bd1832bfb1d_000.obj",
    "00020106_b27a1602d1d44a3d89140ce4_006.obj",
    "00020141_b27a1602d1d44a3d89140ce4_041.obj",
    "00020155_b27a1602d1d44a3d89140ce4_055.obj",
    "00020186_b27a1602d1d44a3d89140ce4_086.obj",
    "00020202_b27a1602d1d44a3d89140ce4_102.obj",
    "00020203_b27a1602d1d44a3d89140ce4_103.obj",
    "00020213_1f65839d7f6c42bf8c2b3391_000.obj"]